# Homework: Vector Search

In [11]:
from fastembed import TextEmbedding
import numpy as np

## Q1. Embeddings the query

Embed the query: `'I just discovered the course. Can I join now?'`. Use the `jinaai/jina-embeddings-v2-small-en` model.

You should get a numpy array of size 512.

What's the minimal value in this array?

In [3]:
model_name = "jinaai/jina-embeddings-v2-small-en"
embedding_model = TextEmbedding(model_name=model_name)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

In [10]:
query = 'I just discovered the course. Can I join now?'
embedding_query = list(embedding_model.embed([query]))[0]
print(f"Size of numpy array: {embedding_query.shape[0]}")
print(f"Minimal value in this array: {embedding_query.min():.3f}")

Size of numpy array: 512
Minimal value in this array: -0.117


In [ ]:
# vectors of embedding model are normalized, so we can use dot product
np.linalg.norm(embedding_query)

1.0

## Q2. Cosine similarity with another vector
Now let's embed this document:
```python
doc = 'Can I still join the course after the start date?'
```
What's the cosine similarity between the vector for the query and the vector for the document?

In [15]:
doc = 'Can I still join the course after the start date?'
embedding_doc = list(embedding_model.embed([doc]))[0]

cosine_similarity = embedding_doc.dot(embedding_query)
print(f"Cosine simialrity between vectors of query and the document: {cosine_similarity:.2f}")

Cosine simialrity between vectors of query and the document: 0.90


## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:
```python
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]
```
Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0)

In [17]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [28]:
text_fields = [record.get('text', '') for record in documents]
embedding_documents = np.array(list((embedding_model.embed(text_fields))))

# calculating cosine similarity
cosine_scores = embedding_documents.dot(embedding_query)
print(f"Document index with the highest similarity: {cosine_scores.argmax()}")

Document index with the highest similarity: 1


## Q4. Ranking by cosine, version two
Now let's calculate a new field, which is a concatenation of question and text:
```python
full_text = doc['question'] + ' ' + doc['text']
```
Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

In [29]:
# concatenation of two fields
full_texts = [record.get('question', '') + ' ' + record.get('text', '') for record in documents]
embedding_documents = np.array(list((embedding_model.embed(full_texts))))

# calculating cosine similarity
cosine_scores = embedding_documents.dot(embedding_query)
print(f"Document index with the highest similarity: {cosine_scores.argmax()}")

Document index with the highest similarity: 0


Is it different from Q3? If yes, why?

Answer: *Yes, the result is different because the question in the document with index 0 contains question semantically close to the query!*

In [ ]:
import re

# get words
words_query = re.findall(r"\w+", query.lower())
words_question = re.findall(r"\w+", documents[0]['question'].lower())

# look at the intersection
set(words_query).intersection(words_question)

{'can', 'course', 'i', 'join', 'the'}

## Q5. Selecting the embedding model
Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

In [54]:
list_of_supported_models = TextEmbedding.list_supported_models()
dimensions = np.array([record['dim'] for record in list_of_supported_models])
print(f"Smallest dimensionality for models in fastembed: {min(dimensions)}")

Smallest dimensionality for models in fastembed: 384


In [59]:
print(f"Supported models with the smallest dimensionality of {min(dimensions)}:\n")
indexes = np.where(dimensions == dimensions.min())[0]
for ind in indexes:
    print(f"    {list_of_supported_models[ind]['model']}")

Supported models with the smallest dimensionality of 384:

    BAAI/bge-small-en
    BAAI/bge-small-en-v1.5
    snowflake/snowflake-arctic-embed-xs
    snowflake/snowflake-arctic-embed-s
    sentence-transformers/all-MiniLM-L6-v2
    sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


One of these models is BAAI/bge-small-en. Let's use it.

## Q6. Indexing with `qdrant` (2 points)
For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:
```python
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```
Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:
```python
text = doc['question'] + ' ' + doc['text']
```

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results? (The score for the first returned record)

In [60]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [78]:
from pprint import pprint
print(f"In the list, there are {len(documents)} documents like that:\n")
pprint(documents[0])

In the list, there are 375 documents like that:

{'course': 'machine-learning-zoomcamp',
 'question': 'How do I sign up?',
 'section': 'General course-related questions',
 'text': 'Machine Learning Zoomcamp FAQ\n'
         'The purpose of this document is to capture frequently asked '
         'technical questions.\n'
         'We did this for our data engineering course and it worked quite '
         'well. Check this document for inspiration on how to structure your '
         'questions and answers:\n'
         'Data Engineering Zoomcamp FAQ\n'
         'In the course GitHub repository there’s a link. Here it is: '
         'https://airtable.com/shryxwLd0COOEaqXo\n'
         'work'}


In [ ]:
from qdrant_client import QdrantClient, models

# Start the client
client = QdrantClient("http://localhost:6333")
client.info()

VersionInfo(title='qdrant - vector search engine', version='1.14.1', commit='530430fac2a3ca872504f276d2c91a5c91f43fa0')

In [ ]:
EMBEDDINGS_DIMENSIONALITY = 384
collection_name = "homework-2"
model_name = "BAAI/bge-small-en"

# Initialize the collection
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDINGS_DIMENSIONALITY,  
        distance=models.Distance.COSINE
    )
)

True

In [70]:
client.collection_exists(collection_name)

True

In [ ]:
# Creating data enitites for Qdrant (points)
points = []

for id, doc in enumerate(documents):
    # When adding the data, use both question and answer fields
    text = doc.get('question', "") + " " + doc.get('text', "")

    point = models.PointStruct(
        id=id,
        vector=models.Document(text=text, model=model_name),
        payload={
            "text": doc['text'],
            "question": doc['question'],
            "section": doc['section'],
            "course": course['course']
        } 
    )
    points.append(point)

In [86]:
points[0]

PointStruct(id=0, vector=Document(text='How do I sign up? Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', model='BAAI/bge-small-en', options=None), payload={'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', 'question': 'How do I sign up?', 'section': 'General course-related qu

In [ ]:
# Index the data
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
# function for vector search
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( 
            text=query,
            model=model_name 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [ ]:
# do the vector search with the query from Question 1
search_result = search(query)

In [ ]:
# highest score is from the best data point found (because of the parameter limit=1) 
best_similarity_score = search_result.points[0].score

print(f'For the query "{query}",\nthe best similarity score of the search results: {best_similarity_score:.2f}')

For the query "I just discovered the course. Can I join now?",
the best similarity score of the search results: 0.87


In [ ]:
# check the metadata of the search result
pprint(search_result.points[0].payload['question'])
pprint(search_result.points[0].payload['text'])

'The course has already started. Can I still join it?'
('Yes, you can. You won’t be able to submit some of the homeworks, but you can '
 'still take part in the course.\n'
 'In order to get a certificate, you need to submit 2 out of 3 course projects '
 'and review 3 peers’ Projects by the deadline. It means that if you join the '
 'course at the end of November and manage to work on two projects, you will '
 'still be eligible for a certificate.')
